# Part 2: Clustering with ClusTCR

ClusTCR is a python package developed to have a fast and accuracte way of clustering large TCR repertoires. ClusTCR uses a 2-step method, first dividing the data into supercluster, before clustering TCR sequences with high sequence similarity. Further information on ClusTCR and all of its possibilities can be found here: https://svalkiers.github.io/clusTCR/

When you use this notebook in google colab, run the first few cells in this notebook to install conda and the ClusTCR package.

If you want to use these notebooks on your local machine, just skip these first three cells and directly import pandas and the clustcr package (after they have been installed locally).

### Start when running in Google Colab

In [1]:
# Check whether conda is already installed
!conda --version

conda 23.3.1


In [ ]:
#If !conda --version returns no results, install conda with :
!pip install -q condacolab
import condacolab 
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:27
🔁 Restarting kernel...


In [ ]:
# Install the clustcr package with conda (can take a little while)
!conda install clustcr -c svalkiers -c bioconda -c pytorch -c conda-forge

### Start when running on local machine

In [2]:
# change your working directory
%cd ..

/home/vincent/Documents/projects/tcr_workshop_2023


In [3]:
# Import packages
import pandas as pd
from clustcr import Clustering

In [ ]:
#Determine the number of threads available
#!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 154
model name	: 12th Gen Intel(R) Core(TM) i7-12800H
stepping	: 3
microcode	: 0x429
cpu MHz		: 803.954
cache size	: 24576 KB
physical id	: 0
siblings	: 20
core id		: 0
cpu cores	: 14
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 32
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb intel_pt sh

In [6]:
# Initiate ClusTCR clustering object
clustering = Clustering(n_cpus=2) # change n_cpus to nunber of threads in your machine

In [7]:
# Load the parsed data from the previous step for clustering
p1_d0 = pd.read_csv('data/P1_0_parsed.tsv', sep='\t', index_col=[0])
p1_d15 = pd.read_csv('data/P1_15_parsed.tsv', sep='\t', index_col=[0])

In [8]:
# Look at the data
p1_d0

,junction_aa,v_call,j_call,Total_count,Total_frequency,productive
0,CASSNSDRTYGDNEQFF,TRBV6-2,TRBJ2-1,33422.0,2.171360e-02,True
1,CATSSVLTQQETQYF,TRBV24-1,TRBJ2-5,24502.0,1.591845e-02,True
2,CASSSRGLANTQYF,TRBV12-3,TRBJ2-3,22361.0,1.452749e-02,True
3,CSVVGADTYEQYF,TRBV29-1,TRBJ2-7,20930.0,1.359780e-02,True
4,CASSLGTALNTEAFF,TRBV7-8,TRBJ1-1,20193.0,1.311898e-02,True
...,...,...,...,...,...,...
99711,CASSPRGDPSTDTQYF,TRBV28,TRBJ2-3,1.0,6.496797e-07,True
99712,CASSLSGTSYEQFF,TRBV27,TRBJ2-1,1.0,6.496797e-07,True
99713,CSATGFSYTEQFF,TRBV20-1,TRBJ2-1,1.0,6.496797e-07,True
99714,CASSVGGGQALWGETQYF,TRBV19,TRBJ2-5,1.0,6.496797e-07,True


In [9]:
# Set a timepoint variable to differentiate between both samples
p1_d0["timepoint"] = "0"
p1_d15["timepoint"] = "15"

data_merged = pd.concat([p1_d0, p1_d15])

In [10]:
# Look at the merged format
data_merged

,junction_aa,v_call,j_call,Total_count,Total_frequency,productive,timepoint
0,CASSNSDRTYGDNEQFF,TRBV6-2,TRBJ2-1,33422.0,2.171360e-02,True,0
1,CATSSVLTQQETQYF,TRBV24-1,TRBJ2-5,24502.0,1.591845e-02,True,0
2,CASSSRGLANTQYF,TRBV12-3,TRBJ2-3,22361.0,1.452749e-02,True,0
3,CSVVGADTYEQYF,TRBV29-1,TRBJ2-7,20930.0,1.359780e-02,True,0
4,CASSLGTALNTEAFF,TRBV7-8,TRBJ1-1,20193.0,1.311898e-02,True,0
...,...,...,...,...,...,...,...
84864,CASSLGSGRSYNEQFF,TRBV10-2,TRBJ2-1,1.0,9.156553e-07,True,15
84865,CASSIDRLVQGLNQPQHF,TRBV19,TRBJ1-5,1.0,9.156553e-07,True,15
84866,CVTCRYPNTEAFF,TRBV6-1,TRBJ1-1,1.0,9.156553e-07,True,15
84867,CASNVVGRLQYF,TRBV28,TRBJ2-7,1.0,9.156553e-07,True,15


In [11]:
# Fit data to the clustering object (+- 5 min)
clustering_result = clustering.fit(data_merged['junction_aa'])

Clustering 179928 TCRs using two-step approach.
Total time to run ClusTCR: 30.694s


In [12]:
# Calculate several pysicochemical features for each cluster (+- 1.5 min)
# The explanation for each feature can be found at: https://svalkiers.github.io/clusTCR/docs/analyzing/features.html
feature_df = clustering_result.compute_features(compute_pgen=False)

/home/vincent/miniforge-pypy3/envs/clustcr_test/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/vincent/miniforge-pypy3/envs/clustcr_test/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Show the features per cluster
feature_df

,h,size,length,basicity_avg,hydrophobicity_avg,helicity_avg,mutation stability_avg,basicity_var,hydrophobicity_var,helicity_var,mutation stability_var
cluster,,,,,,,,,,,
0,0.128411,463,11,210.391643,-0.460184,1.044770,19.468018,1.162209,0.221931,0.000798,1.955451
1,0.140781,213,11,211.003142,-0.237526,1.046844,19.911520,0.939708,0.181185,0.000607,1.762398
2,0.149787,182,11,209.969527,-0.320558,1.053339,21.358833,1.600075,0.264130,0.000937,2.007210
3,0.155164,179,11,211.394327,-0.491749,1.026721,19.295660,1.776918,0.201644,0.000957,1.753262
4,0.164714,23,11,210.155184,-0.145318,1.051806,19.943144,0.389686,0.164665,0.000560,2.013565
...,...,...,...,...,...,...,...,...,...,...,...
7100,0.071429,2,14,212.031250,-2.138125,0.983437,17.531250,0.001953,0.000253,0.000024,0.001953
7101,0.071429,2,14,213.293750,-1.719062,1.002500,19.093750,0.894453,0.003150,0.000020,0.048828
7102,0.071429,2,14,210.793750,-1.663750,1.010000,18.062500,0.041328,0.110450,0.000200,1.125000


In [ ]:
# Get a summary table depicting the cluster motif, cluster size, and all TCRs within the cluster
clustering_summary = clustering_result.summary()
clustering_summary['sequences'] = clustering_result.cluster_contents()

# Display summary data
clustering_summary

,size,motif,sequences
0,463,CASS...TDTQYF,"[CASSGGSTDTQYF, CASSGSSTDTQYF, CASSASSTDTQYF, ..."
1,213,CASS...QETQYF,"[CASSFGTQETQYF, CASSIGTQETQYF, CASSLTLKETQYF, ..."
2,182,CASS...GDEQFF,"[CASSLAGGDEQYF, CASSLFGGDEQYF, CASSLIRGDEQFF, ..."
3,179,CASS....NEQFF,"[CASSLARDNEQFF, CASSLVNSNEQFF, CASSLVVSNEQFF, ..."
4,23,CASSl.g[SD]NEQFF,"[CASSLVRDNEQFF, CASSFTGGNEQFF, CASSFTGSNEQFF, ..."
...,...,...,...
7100,2,CASSQD[DS]RGGNQPQHF,"[CASSQDDRGGNQPQHF, CASSQDSRGGNQPQHF]"
7101,2,CASTRGG[RE]YSNQPQHF,"[CASTRGGEYSNQPQHF, CASTRGGRYSNQPQHF]"
7102,2,CASSQE[LQ]GAGNQPQHF,"[CASSQELGAGNQPQHF, CASSQEQGAGNQPQHF]"
7103,2,CASSDRR[QS]SYNSPLHF,"[CASSDRRQSYNSPLHF, CASSDRRSSYNSPLHF]"


In [ ]:
# Display sequence information
clustering_clusters = clustering_result.clusters_df
clustering_clusters

# Save results as file for in part 4 of this tutorial
clustering_clusters.to_csv("data/clustcr_results/clustcr_results.tsv", sep="\t")

NameError: name 'clustering_result' is not defined

## Comparing clusters between repertoires
This same clustering analysis can be performed for both repertoires separately.
Then you can compare clusters, cluster sizes, features, ... between both timepoints.

However, comparing clusters and cluster features can be performed much more efficient using the code provided below. Here, we divide the clusters per sample (P1_0 and P1_15) and compare cluster size, clonal count and clonal frequency per cluster and per sample.

In [ ]:
# Add the cluster numbers and features to the original data
data_merged = pd.merge(
    left = data_merged,
    right= clustering_clusters,
    on="junction_aa",
    how="right"
)

# Show the table
data_merged

,junction_aa,v_call,j_call,Total_count,Total_frequency,timepoint,cluster,basicity,hydrophobicity,helicity,mutation stability
0,CASSGGSTDTQYF,TRBV10-2,TRBJ2-3,7.0,0.000004,0,0,208.630769,-0.993846,1.056154,19.615385
1,CASSGGSTDTQYF,TRBV10-1,TRBJ2-3,8.0,0.000007,15,0,208.630769,-0.993846,1.056154,19.615385
2,CASSGSSTDTQYF,TRBV6-4,TRBJ2-3,2.0,0.000001,0,0,209.007692,-0.958462,1.044615,18.307692
3,CASSASSTDTQYF,TRBV2,TRBJ2-3,7.0,0.000004,0,0,209.292308,-0.691538,1.051538,17.230769
4,CASSPSSTDTQYF,TRBV12-3,TRBJ2-3,9.0,0.000006,0,0,209.907692,-1.082308,1.000000,17.769231
...,...,...,...,...,...,...,...,...,...,...,...
68572,CASSQEQGAGNQPQHF,TRBV4-1,TRBJ1-5,10.0,0.000006,0,7102,210.937500,-1.898750,1.000000,17.312500
68573,CASSDRRQSYNSPLHF,TRBV6-1,TRBJ1-6,5.0,0.000003,0,7103,214.093750,-1.319375,0.994375,18.625000
68574,CASSDRRSSYNSPLHF,TRBV5-1,TRBJ1-6,2.0,0.000002,15,7103,213.681250,-1.325000,0.996875,18.625000
68575,CASSKRQGSSYNSPLHF,TRBV5-5,TRBJ1-6,3.0,0.000002,0,7104,212.470588,-1.588824,1.005882,19.470588


In [ ]:
# Compare cluster size, clonotype count and frequency between different clusters

# step 1: group the data per cluster number and timepoint
# step 2: get the number of TCRs in each cluster with '"junction_aa":len'
# step 3: sum all individual clone counts and frequencies per cluster to get a cluster total
# step 4: rename the columns and fill NaN fields
cluster_sizes = (
    data_merged
    .groupby(["cluster", "timepoint"])
    .agg({"junction_aa":len, "Total_count":sum, "Total_frequency":sum})
    .rename({"junction_aa":"size"}, axis="columns")
    .unstack()
    .fillna(0)
    )


In [ ]:
# Calcualte the difference in cluster size between day 0 and day 15
cluster_sizes['size', 'delta'] = cluster_sizes['size', '15'] - cluster_sizes['size', '0']
cluster_sizes['size', 'delta'] = cluster_sizes['size', '15'] - cluster_sizes['size', '0']

In [ ]:
# Show comparison data for cluster size (and change in size), clonal count and clonal frequency per cluster
cluster_sizes

size        Total_count         Total_frequency            size
timepoint      0     15           0      15               0        15 delta
cluster                                                                    
0          301.0  235.0      3750.0  4399.0        0.002396  0.003967 -66.0
1          129.0  103.0      1302.0   847.0        0.000832  0.000764 -26.0
2          100.0   89.0       931.0   741.0        0.000595  0.000668 -11.0
3          121.0   71.0      1410.0   637.0        0.000901  0.000574 -50.0
4           13.0   11.0       119.0    80.0        0.000076  0.000072  -2.0
...          ...    ...         ...     ...             ...       ...   ...
7100         0.0    2.0         0.0    10.0        0.000000  0.000009   2.0
7101         2.0    0.0        18.0     0.0        0.000012  0.000000  -2.0
7102         1.0    1.0        10.0     9.0        0.000006  0.000008   0.0
7103         1.0    1.0         5.0     2.0        0.000003  0.000002   0.0
7104         1.0    1.0         3.0     2.0        0.000002  0.000002   0.0

[7105 rows x 7 columns]

In [ ]:
# Sort the data to find which cluster sizes changed the most between both time points
cluster_sizes["size"].sort_values(by="delta")

timepoint,0,15,delta
cluster,,,
4611,362.0,269.0,-93.0
5116,229.0,154.0,-75.0
1446,233.0,165.0,-68.0
0,301.0,235.0,-66.0
5131,200.0,139.0,-61.0
...,...,...,...
6655,32.0,45.0,13.0
1494,3.0,16.0,13.0
4200,66.0,82.0,16.0
